In [1]:
%%capture
pip install -U CwnGraph

In [2]:
# load CWN data (https://lopentu.github.io/CwnWeb)
import CwnGraph
CwnGraph.download()
from CwnGraph import CwnBase
cwn = CwnBase()

In [3]:
import requests
from bs4 import BeautifulSoup
import itertools

In [4]:
def get_relations(word):

  # CWN (https://lopentu.github.io/CwnWeb)
  lemma = cwn.find_lemma(f'^{word}$')
  sense = [l.senses for l in lemma]
  sense = list(itertools.chain.from_iterable(sense)) # flattern the list

  rel = [s.relations for s in sense]
  rel = list(itertools.chain.from_iterable(rel))
  rel_word = list(set( [(x[0], x[1].head_word) for x in rel if 'Synset' not in str(x[1])] ))
  
  cwn_relations = dict()
  for rel,word in rel_word:
    cwn_relations.setdefault(rel, []).append(word)

  # MOE dict (https://www.moedict.tw)
  res = requests.get(url = f'https://www.moedict.tw/{word}')
  soup = BeautifulSoup(res.text, 'lxml')

  span_ant = soup.select('span.antonyms')
  span_syn = soup.select('span.synonyms')

  if span_ant:
    ant = soup.select('span.antonyms')[0].get_text()
    ants = ant[1:].split('、')
  else:
    ants = []
  
  if span_syn:
    syn = soup.select('span.synonyms')[0].get_text()
    syns = syn[1:].split('、')
  else: syns = []

  moe_relations = {'antonyms': ants, 'synonyms': syns}

  semantic_relations = {'CWN': cwn_relations, 'MOE': moe_relations}

  return semantic_relations

In [6]:
get_relations('容忍')

{'CWN': {'has_facet': ['容忍']},
 'MOE': {'antonyms': ['發作', '拒絕', '生氣'], 'synonyms': ['忍耐']}}